In [41]:


dir_image = "../train_data-PLEIADES-972-2022/images/"
dir_label = "../train_data-PLEIADES-972-2022/labels/"
print(len(os.listdir(dir_label)))
num_image = 4567

name_image = os.listdir(dir_image)[num_image]
file_image = dir_image + name_image
name_label = name_image.split(".")[0] + ".npy"

file_label = dir_label +name_label


In [42]:
SatelliteImage.from_raster(
    file_image,
    "973",
    "",
    3
).plot([0,1,2])

label_mask = np.load(file_label)

plt.imshow(label_mask, cmap='gray', vmin=0, vmax=1)
plt.axis('off')
plt.show()

## Test des dataloader

In [4]:
%load_ext autoreload
%autoreload 2
import os
os.getcwd()

In [3]:
import os
os.chdir("src")
import matplotlib.pyplot as plt
from classes.data.satellite_image import SatelliteImage
import numpy as np
import gc
import os
import sys
from datetime import datetime
from tqdm import tqdm
import mlflow
import numpy as np
import pytorch_lightning as pl
import torch
import yaml
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from torch.utils.data import DataLoader
from yaml.loader import SafeLoader
#from classes.optim.losses import CrossEntropySelfmade
from classes.data.satellite_image import SatelliteImage
from classes.labelers.labeler import BDTOPOLabeler, RILLabeler
#from classes.optim.losses import CrossEntropySelfmade
from torch.nn import CrossEntropyLoss
from classes.optim.optimizer import generate_optimization_elements
from data.components.dataset import PleiadeDataset
from models.components.segmentation_models import DeepLabv3Module
from models.segmentation_module import SegmentationModule
from train_pipeline_utils.download_data import load_satellite_data, load_donnees_test
from train_pipeline_utils.handle_dataset import (
    generate_transform,
    select_indices_to_split_dataset
)

from train_pipeline_utils.prepare_data import(
    filter_images,
    label_images,
    save_images_and_masks,
    check_labelled_images
)

from classes.data.satellite_image import SatelliteImage
from classes.data.labeled_satellite_image import SegmentationLabeledSatelliteImage
from utils.utils import update_storage_access, split_array, remove_dot_file
from rasterio.errors import RasterioIOError
from classes.optim.evaluation_model import evaluer_modele_sur_jeu_de_test_segmentation_pleiade

def download_data(config):
    """
    Downloads data based on the given configuration.

    Args:
        config: a dictionary representing the 
        configuration information for data download.

    Returns:
        A list of output directories for each downloaded dataset.
    """

    print("Entre dans la fonction download_data")
    config_data = config["donnees"]
    list_output_dir = []
    list_masks_cloud_dir = []
    
    years = config_data["year"]
    deps = config_data["dep"]
    src = config_data["source train"]

    for year, dep in zip(years, deps):
        # year, dep = years[0], deps[0]
        if src == "PLEIADES":
            cloud_dir = load_satellite_data(year, dep, "NUAGESPLEIADES")
            list_masks_cloud_dir.append(cloud_dir)

        output_dir = load_satellite_data(year, dep, src)
        list_output_dir.append(output_dir)
    
    print("chargement des données test")
    test_dir = load_donnees_test(type=config["donnees"]["task"])

    return list_output_dir, list_masks_cloud_dir, test_dir



def prepare_train_data(config, list_data_dir, list_masks_cloud_dir):
    """
    Preprocesses and splits the raw input images 
    into tiles and corresponding masks, 
    and saves them in the specified output directories. 
    
    Args:
        config: A dictionary representing the configuration settings.
        list_data_dir: A list of strings representing the paths
        to the directories containing the raw input image files.
    
    Returns:
        A list of strings representing the paths to 
        the output directories containing the 
        preprocessed tile and mask image files.
    """

    print("Entre dans la fonction prepare_data")
    config_data = config["donnees"]

    years = config_data["year"]
    deps = config_data["dep"]
    src = config_data["source train"]
    labeler = config_data["type labeler"]

    list_output_dir = []

    for i, (year, dep) in enumerate(zip(years, deps)):
        # i, year , dep = 0,years[0],deps[0]
        date = datetime.strptime(str(year) + "0101", "%Y%m%d")
        if labeler == "RIL":
            buffer_size = config_data["buffer size"]
            labeler = RILLabeler(date, dep=dep, buffer_size=buffer_size)
        elif labeler == "BDTOPO":
            labeler = BDTOPOLabeler(date, dep=dep)

        output_dir = (
            "../train_data" + "-" + src + "-" + dep + "-" + str(year) + "/"
        )

        if not check_labelled_images(output_dir):
            
            if src == "PLEIADES":
                cloud_dir = list_masks_cloud_dir[i]
                list_name_cloud = [path.split("/")[-1].split(".")[0] for path in os.listdir(cloud_dir)]
            
            dir = list_data_dir[i]
            list_path = [dir + "/" + filename for filename in os.listdir(dir)]
            
            for path in tqdm(list_path):
                # path = list_path[0]
                # path  = dir + "/"+ "ORT_2022_0691_1641_U20N_8Bits.jp2"
                try:
                    si = SatelliteImage.from_raster(
                        file_path=path,
                        dep=dep,
                        date=date,
                        n_bands=config_data["n channels train"]
                    )
                    
                except RasterioIOError:
                    print("Erreur de lecture du fichier " + path)
                    continue

                else:
                    filename = path.split("/")[-1].split(".")[0] 
                    list_splitted_mask_cloud = None

                    if filename in list_name_cloud:
                        mask_full_cloud = np.load(cloud_dir + "/" + filename + ".npy")
                        list_splitted_mask_cloud = split_array(mask_full_cloud, config_data["tile size"])
                        
                    list_splitted_images = si.split(config_data["tile size"]) 
                    
                    list_filtered_splitted_images = filter_images(
                        config_data["source train"],
                        list_splitted_images,
                        list_splitted_mask_cloud 
                    )

                    list_filtered_splitted_labeled_images, list_masks = label_images(
                        list_filtered_splitted_images, labeler
                    )

                    save_images_and_masks(
                        list_filtered_splitted_labeled_images, list_masks, output_dir
                    )

        list_output_dir.append(output_dir)

    return list_output_dir


def prepare_test_data(config,test_dir):

    images_path = test_dir + "/images"
    labels_path = test_dir + "/masks"

    list_name_image = os.listdir(images_path)
    list_name_label = os.listdir(labels_path)

    list_name_image = np.sort(remove_dot_file(list_name_image))
    list_name_label = np.sort(remove_dot_file(list_name_label))

    list_images_path = [images_path + "/" + name for name in list_name_image]
    list_labels_path = [labels_path + "/" + name for name in list_name_label]
    
    output_test = "../test-data"
    output_images_path = output_test + "/images"
    output_labels_path = output_test + "/labels"

    n_bands = config["donnees"]["n bands"]
    tile_size = config["donnees"]["tile size"]
    
    if not os.path.exists(output_labels_path):
        os.makedirs(output_labels_path)
    else:
        return None

    for image_path, label_path, name in zip(
        list_images_path,
        list_labels_path,
        list_name_image
        ):

        si = SatelliteImage.from_raster(
            file_path=image_path, dep=None, date=None, n_bands=n_bands
        )
        mask = np.load(label_path)

        lsi = SegmentationLabeledSatelliteImage(si, mask, "", "")
        list_lsi = lsi.split(tile_size)

        for i, lsi in enumerate(list_lsi):
            file_name_i = name.split(".")[0] + "_" + "{:03d}".format(i)
            
            lsi.satellite_image.to_raster(
                output_images_path, file_name_i + ".jp2"
                )
            np.save(output_labels_path + "/" + file_name_i + ".npy", lsi.label)


def instantiate_dataset(config, list_path_images, list_path_labels):
    """
    Instantiates the appropriate dataset object
    based on the configuration settings.
    
    Args:
        config: A dictionary representing the configuration settings.
        list_path_images: A list of strings representing
        the paths to the preprocessed tile image files.
        list_path_labels: A list of strings representing
        the paths to the corresponding preprocessed mask image files.
    
    Returns:
        A dataset object of the specified type.
    """
    dataset_dict = {"PLEIADE": PleiadeDataset}
    dataset_type = config["donnees"]["dataset"]

    # inqtanciation du dataset comple
    if dataset_type not in dataset_dict:
        raise ValueError("Invalid dataset type")
    else:
        full_dataset = dataset_dict[dataset_type](
            list_path_images, list_path_labels
        )

    return full_dataset


def instantiate_dataloader(config, list_output_dir):
    """
    Instantiates and returns the data loaders for
    training, validation, and testing datasets.
    
    Args:
    - config (dict): A dictionary containing the configuration parameters
    for data loading and processing.
    - list_output_dir (list): A list of strings containing the paths to
    the directories that contain the training data.
    
    Returns:
    - train_dataloader (torch.utils.data.DataLoader):
    The data loader for the training dataset.
    - valid_dataloader (torch.utils.data.DataLoader): 
    The data loader for the validation dataset.
    - test_dataloader (torch.utils.data.DataLoader): 
    The data loader for the testing dataset.
    
    The function first generates the paths for the image and label data
    based on the data source (Sentinel, PLEIADES) vs pre-annotated datasets.
    It then instantiates the required dataset class
    (using the `intantiate_dataset` function) and splits the full dataset
    into training and validation datasets based on the validation proportion
    specified in the configuration parameters.
    
    Next, the appropriate transformations are applied to the training
    and validation datasets using the `generate_transform` function.
    
    Finally, the data loaders for the training and validation datasets 
    are created using the `DataLoader` class from the PyTorch library, 
    and the data loader for the testing dataset is set to `None`.
    """
    # génération des paths en fonction du type de Données
    # (Sentinel, PLEIADES) VS Dataset préannotés

    if config["donnees"]["source train"] in ["PLEIADES", "SENTINEL2"]:
        list_path_labels = []
        list_path_images = []
        for dir in list_output_dir:
            # dir = list_output_dir[0]
            labels = os.listdir(dir + "/labels") 
            images = os.listdir(dir + "/images")

            list_path_labels = np.concatenate((
                list_path_labels,
                np.sort([dir + "/labels/" + name for name in labels])
            ))
            
            list_path_images = np.concatenate((
                list_path_images,
                np.sort([dir + "/images/" + name for name in images])
            ))

    train_idx, val_idx = select_indices_to_split_dataset(
        len(list_path_images),
        config["optim"]["val prop"]
     )
    
    # récupération de la classe de Dataset souhaitée
    train_dataset = instantiate_dataset(
        config, list_path_images[train_idx], list_path_labels[train_idx]
    )

    valid_dataset = instantiate_dataset(
        config, list_path_images[val_idx], list_path_labels[val_idx]
    )

    # on applique les transforms respectives
    augmentation = config["donnees"]["augmentation"]
    tile_size = config["donnees"]["tile size"]
    t_aug, t_preproc = generate_transform(tile_size, augmentation)
    train_dataset.transforms = t_aug
    valid_dataset.transforms = t_preproc

    # création des dataloader
    batch_size = config["optim"]["batch size"]

    train_dataloader, valid_dataloader = [
        DataLoader(
            ds,
            batch_size=batch_size,
            shuffle=boolean,
            num_workers=2,
        )
        for ds, boolean in zip([train_dataset, valid_dataset], [True, False])
    ]

    # Gestion datset test
    output_test = "../test-data"
    output_images_path = output_test + "/images/"
    output_labels_path = output_test + "/labels/"

    list_name_image = os.listdir(output_images_path)
    list_name_label = os.listdir(output_labels_path)

    list_path_images = np.sort([output_images_path + name_image for name_image in list_name_image])
    list_path_labels = np.sort([output_labels_path + name_label for name_label in list_name_label])

    dataset_test = instantiate_dataset(
        config, list_path_images, list_path_labels
    )
    
    dataset_test.transforms = t_preproc
    
    batch_size_test = config["optim"]["batch size test"]
    test_dataloader = DataLoader(
            dataset_test,
            batch_size=batch_size_test,
            shuffle=False,
            num_workers=2,
        )
    
    return train_dataloader, valid_dataloader, test_dataloader


def instantiate_model(config):
    """
    Instantiate a module based on the provided module type.

    Args:
        module_type (str): Type of module to instantiate.

    Returns:
        object: Instance of the specified module.
    """
    module_type = config["optim"]["module"]
    module_dict = {"deeplabv3": DeepLabv3Module}
    nchannel = config["donnees"]["n channels train"]

    if module_type not in module_dict:
        raise ValueError("Invalid module type")

    if module_type == "deeplabv3":
        return module_dict[module_type](nchannel)
    else:
        return module_dict[module_type]()


def instantiate_loss(config):
    """
    intantiates an optimizer object with the parameters 
    specified in the configuration file.

    Args:
        model: A PyTorch model object.
        config: A dictionary object containing the configuration parameters.

    Returns:
        An optimizer object from the `torch.optim` module.
    """
    loss_type = config["optim"]["loss"]
    loss_dict = {
                "crossentropy": CrossEntropyLoss,
                "crossentropyselmade": CrossEntropySelfmade
                }

    if loss_type not in loss_dict:
        raise ValueError("Invalid loss type")
    else:
        return loss_dict[loss_type]()


def instantiate_lightning_module(config, model):
    """
    Create a PyTorch Lightning module for segmentation
    with the given model and optimization configuration.

    Args:
        config (dict): Dictionary containing the configuration
        parameters for optimization.
        model: The PyTorch model to use for segmentation.

    Returns:
        A PyTorch Lightning module for segmentation.
    """
    list_params = generate_optimization_elements(config)

    lightning_module = SegmentationModule(
        model=model,
        loss=instantiate_loss(config),
        optimizer=list_params[0],
        optimizer_params=list_params[1],
        scheduler=list_params[2],
        scheduler_params=list_params[3],
        scheduler_interval=list_params[4],
    )
    return lightning_module


def instantiate_trainer(config, lightning_module):
    """
    Create a PyTorch Lightning module for segmentation with 
    the given model and optimization configuration.

    Args:
        config (dict): Dictionary containing the configuration
        parameters for optimization.
        model: The PyTorch model to use for segmentation.

    Returns:
        SegmentationModule: A PyTorch Lightning module for segmentation.
    """
    # def callbacks
    checkpoint_callback = ModelCheckpoint(
        monitor="validation_loss", save_top_k=1, save_last=True, mode="max"
    )
    early_stop_callback = EarlyStopping(
        monitor="validation_loss", mode="max", patience=3
    )
    lr_monitor = LearningRateMonitor(logging_interval="step")
    list_callbacks = [lr_monitor, checkpoint_callback, early_stop_callback]

    strategy = "auto"

    trainer = pl.Trainer(
        callbacks=list_callbacks,
        max_epochs=config["optim"]["max epochs"],
        num_sanity_val_steps=2,
        strategy=strategy,
        log_every_n_steps=2,
    )

    return trainer


with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

list_data_dir, list_masks_cloud_dir, test_dir = download_data(config)

# list_data_dir = ["../data/PLEIADES/2022/MARTINIQUE"]
# list_masks_cloud_dir = ["../data/NUAGESPLEIADES/2022/MARTINIQUE"]

list_output_dir = prepare_train_data(config, list_data_dir, list_masks_cloud_dir)
prepare_test_data(config, test_dir)

model = instantiate_model(config)

train_dl, valid_dl, test_dl = instantiate_dataloader(
    config, list_output_dir
)

In [4]:
print(len(train_dl.dataset))
len(valid_dl.dataset)

In [16]:
train_ds = train_dl.dataset
num_element = 5
num_element = 5000
num_element = 10000
num_element = 12000
num_element = 3000


In [17]:

images, labels, dic = train_ds[num_element]
num_dans_batch = 0
print(dic["pathlabel"],dic["pathimage"])    
print(images)

SatelliteImage.from_raster(
    dic["pathimage"],
    "973",
    "",
    3
).plot([0,1,2])

# mask mis pour l'entrainement
plt.imshow(labels, cmap='gray', vmin=0, vmax=1)
plt.axis('off')
plt.show()

# path du mask
label_mask = np.load(dic["pathlabel"])

plt.imshow(label_mask, cmap='gray', vmin=0, vmax=1)
plt.axis('off')
plt.show()

del images, labels, dic
